In [46]:
import openmm as mm
import simtk
import numpy as np

from polykit.polykit.generators.initial_conformations import grow_cubic, create_random_walk

In [39]:
platform_object = mm.Platform.getPlatformByName("CPU")

In [55]:
import logging
import time

logging.basicConfig(level=logging.INFO)


class Simulation(object):
    def __init__(self, **kwargs):
        """
        Initialize a simulation object.

        Parameters
        ----------
        integrator_type : str, optional
            Type of integrator to use. Default is 'Langevin'.
        temperature : float, optional
            Temperature in reduced units. Default is 1.0.
        gamma : float, required
            Friction/damping constant in units of reciprocal time (1/τ). Default is 0.1.
        timestep : float, required
            Simulation time step in units of τ. Default is 0.005.
        platform : str, required
            Platform to use. Default is 'CPU'.
            Options are 'CUDA', 'OpenCL', or 'Reference'.
        reporter : HDF5Reporter, optional
            Reporter object for saving simulation data. If None, no data will be saved.
        """


        #integrator parameters
        self.integrator_type = kwargs.get("integrator_type", "Langevin")
        self.temperature = kwargs.get("temperature", 1.0)
        self.gamma = kwargs.get("gamma", 0.1) #units of 1/tau
        self.timestep = kwargs.get("timestep", 0.005) #units of tau

        #platform parameters
        self.platform = kwargs.get("platform", "CPU")
        
        #system parameters
        self.system = mm.System()

        #global parameters
        self.N = kwargs.get("N")
        self.kB = simtk.unit.BOLTZMANN_CONSTANT_kB * simtk.unit.AVOGADRO_CONSTANT_NA
        self.kT = self.temperature * self.kB
        self.conlen = kwargs.get("conlen", 1.0)

        #Internal state
        self.positions = None
        self.velocities = None
        self.applied_forces = None
        self.step = 0
        self.block = 0
        self.time = 0

        #setup openmm system
        self.create_system_object()
        self.create_integrator_object()
        self.create_platform_object()

        #initialize reporter
        self.reporter = kwargs.get("reporter", None)
        
        assert self.temperature is not None, "Temperature must be specified."
        assert self.N is not None, "N must be specified."
    #need to add topology
    
    def set_positions(self, positions, center=np.zeros(3), random_offset=1e-5):
        """
        Set the positions of particles in the simulation, with centering and a small random offset.

        Parameters
        ----------
        positions : numpy.ndarray
            An Nx3 array of particle positions (in reduced units).
        center : array-like, optional
            The coordinate to center the positions around (default: [0, 0, 0]).
        random_offset : float, optional
            Magnitude of random offset to add to each coordinate (default: 1e-5).

        Returns
        -------
        None
        """
        # Validate input
        if positions.shape[0] != self.N:
            raise ValueError(f"Expected {self.N} particles, got {positions.shape[0]}")
        if positions.shape[1] != 3:
            raise ValueError("Positions must be Nx3 array")

        # Center positions
        centroid = np.mean(positions, axis=0)
        pos_centered = positions - centroid + np.asarray(center)

        # Add small random offset
        pos_final = pos_centered + np.random.uniform(-random_offset, random_offset, pos_centered.shape)

        self.positions = pos_final

    def set_velocities(self):
        """
        Set initial velocities according to Maxwell-Boltzmann distribution at the specified temperature.
        """
        if not hasattr(self, 'context'):
            raise RuntimeError("Context must be created before setting velocities")
        
        # Get the mass of the first particle (all particles have same mass in our case)
        mass = self.system.getParticleMass(0)
        
        # Calculate standard deviation based on temperature and mass
        # Use reduced units (temperature is already in reduced units)
        sigma = np.sqrt(self.temperature / mass._value)
        
        # Generate random velocities from normal distribution
        velocities = np.random.normal(0, sigma, size=(self.N, 3))
        
        # Set velocities in the context
        self.context.setVelocities(velocities)
    
    def add_force(self, force):
        """
        Add a force to the system.

        Parameters
        ----------
        force : mm.Force
            The force to add.

        Returns
        -------
        int
            Index of the added force.
        """
        return self.system.addForce(force)
    
    def run_simulation_block(
        self,
        steps=None,
        check_functions=[],
        get_velocities=False,
        save=True,
        save_extras={},
    ):
        """
        Perform one block of simulation steps.

        Parameters
        ----------
        steps : int or None
            Number of timesteps to perform. If None, uses default steps.
        check_functions : list of functions, optional
            List of functions to call every block. Coordinates are passed to each function.
            If any function returns False, simulation is stopped.
        get_velocities : bool, default=False
            If True, will return velocities in the result.
        save : bool, default=True
            If True, save results of this block.
        save_extras : dict
            Additional information to save with the results.

        Returns
        -------
        dict
            Dictionary containing simulation results including positions, energies, and time.
        """
        # Set default steps if not provided
        if steps is None:
            steps = 1000  # Default steps per block

        # Perform integration steps
        start_time = time.time()
        self.integrator.step(steps)
        end_time = time.time()

        # Get state information
        self.state = self.context.getState(
            getPositions=True,
            getVelocities=get_velocities,
            getEnergy=True
        )

        # Extract coordinates and convert to numpy array
        coords = self.state.getPositions(asNumpy=True)
        coords = np.array(coords, dtype=np.float32)

        # Calculate energies per particle (in units of kT)
        eK = self.state.getKineticEnergy()._value / self.N / self.kT._value
        eP = self.state.getPotentialEnergy()._value / self.N / self.kT._value
        
        # Get time in reduced units (τ)
        curtime = self.state.getTime()._value  # This gives time in reduced units

        # Calculate steps per second
        steps_per_second = steps / (end_time - start_time)

        # Log simulation progress
        msg = f"block {self.block:4d} "
        msg += f"pos[1]=[{coords[0][0]:.1f} {coords[0][1]:.1f} {coords[0][2]:.1f}] "
        msg += f"t={curtime:.1f}τ "  # Changed to show time in reduced units
        msg += f"kin={eK:.2f} pot={eP:.2f} "
        msg += f"SPS={steps_per_second:.0f}"

        logging.info(msg)

        # Run check functions if provided
        check_fail = False
        for check_function in check_functions:
            if not check_function(coords):
                check_fail = True
                break

        # Basic error checks
        if np.isnan(coords).any():
            raise RuntimeError("Coordinates contain NaN values")
        if np.isnan(eK) or np.isnan(eP):
            raise RuntimeError("Energy values contain NaN")
        if check_fail:
            raise RuntimeError("Custom checks failed")

        # Prepare result dictionary
        result = {
            "pos": coords,
            "potentialEnergy": eP,
            "kineticEnergy": eK,
            "time": curtime,
            "block": self.block,
        }

        # Add velocities if requested
        if get_velocities:
            result["vel"] = self.state.getVelocities(asNumpy=True)

        # Add any extra information
        result.update(save_extras)

        # Save results using reporter if available and save is True
        if self.reporter is not None and save:
            self.reporter.report("data", result)

        # Update simulation state
        self.block += 1
        self.step += steps
        self.time = curtime

        return result
    
    def save_initial_state(self):
        """
        Save initial simulation state using the reporter if available.
        """
        if self.reporter is not None:
            # Save initial arguments
            init_args = {
                "integrator_type": self.integrator_type,
                "temperature": self.temperature,
                "gamma": self.gamma,
                "timestep": self.timestep,
                "platform": self.platform,
                "N": self.N,
                "conlen": self.conlen
            }
            self.reporter.report("initArgs", init_args)
            
            # Save starting conformation
            if self.positions is not None:
                self.reporter.report("starting_conformation", {"pos": self.positions})

    def print_stats(self):
        """
        Print current simulation statistics.
        """
        if not hasattr(self, 'state'):
            print("No simulation state available")
            return

        # Get current state
        self.state = self.context.getState(getEnergy=True)
        
        # Calculate energies per particle (in units of kT)
        eK = self.state.getKineticEnergy()._value / self.N / self.kT._value
        eP = self.state.getPotentialEnergy()._value / self.N / self.kT._value
        total_energy = eK + eP
        
        # Get time in reduced units
        curtime = self.state.getTime()._value

        # Print statistics
        print("\nSimulation Statistics:")
        print(f"Current block: {self.block}")
        print(f"Total steps: {self.step}")
        print(f"Simulation time: {curtime:.2f}τ")
        print(f"Kinetic energy: {eK:.2f} kT/particle")
        print(f"Potential energy: {eP:.2f} kT/particle")
        print(f"Total energy: {total_energy:.2f} kT/particle")
    

    def create_system_object(self):
        self.system = mm.System()
        #add particles to system
        for _ in range(self.N):
            self.system.addParticle(1.0)  # Add particles with mass 1.0

    def create_integrator_object(self):
        if self.integrator_type == 'Langevin':
            self.integrator = mm.LangevinIntegrator(
                self.temperature,
                self.gamma,
                self.timestep
            )
        elif self.integrator_type == 'variableLangevin':
            self.integrator = mm.VariableLangevinIntegrator(
                self.temperature,
                self.gamma,
                self.timestep
            )

    def create_platform_object(self):
        if self.platform == 'CUDA':
            self.platform_object = mm.Platform.getPlatformByName('CUDA')
            self.platform_properties = {'CudaPrecision': 'double'}
        elif self.platform == 'OpenCL':
            self.platform_object = mm.Platform.getPlatformByName('OpenCL')
            self.platform_properties = {'OpenCLPrecision': 'double'}
        elif self.platform == 'Reference':
            self.platform_object = mm.Platform.getPlatformByName('Reference')
        elif self.platform == 'CPU':
            self.platform_object = mm.Platform.getPlatformByName('CPU')
        else:
            print("platform_type can be either CUDA, OpenCL, or CPU")

    def create_context(self):
        self.context = mm.Context(self.system, self.integrator, self.platform_object)
        self.context.setPositions(self.positions)
    

In [48]:
class Chromosome(object):
    def __init__(self, N, chains, sim_object, extra_bonds=None, extra_triplets=None):
        """
        Initialize a Chromosome object and generate bond/angle lists.

        Parameters
        ----------
        N : int
            Total number of particles.
        chains : list of tuples
            List of (start, end, isRing) tuples defining chain segments.
        sim_object : Simulation
            Simulation object, must have attribute `N`.
        extra_bonds : list of tuples, optional
            Additional (i, j) bonds.
        extra_triplets : list of tuples, optional
            Additional (i, j, k) angle triplets.
        """
        self.N = N
        self.chains = chains
        self.sim_object = sim_object

        self.bond_list = self._generate_bonds(sim_object, chains, extra_bonds)
        self.triplet_list = self._generate_triplets(sim_object, chains, extra_triplets)
        
    def add_harmonic_bond(self, force_group=0, k=30.0, r0=1.5):
        """
        Create a HarmonicBondForce from self.bond_list.

        Parameters
        ----------
        force_group : int
            Force group ID.
        k : float
            Spring constant (reduced units).
        r0 : float
            Equilibrium bond distance.

        Returns
        -------
        mm.HarmonicBondForce
        """
        bond_force = mm.HarmonicBondForce()
        bond_force.setForceGroup(force_group)
        for idx1, idx2 in self.bond_list:
            bond_force.addBond(int(idx1), int(idx2), r0, k)
        return bond_force
    
    def add_angle_force(self, k=1.5, theta_0=np.pi, force_group=1, override_checks=False):
        """
        Add harmonic angle force: U(θ) = 0.5 * k * (θ - θ₀)² for each triplet.

        Parameters
        ----------
        k : float or list
            Stiffness (unitless, in kT). Scalar or per-triplet.
        theta_0 : float or list
            Equilibrium angle(s), in radians. Scalar or per-triplet.
        force_group : int
            OpenMM force group ID.
        override_checks : bool
            Skip duplicate triplet checks.

        Returns
        -------
        mm.CustomAngleForce
        """
        if not override_checks:
            self._check_angle_bonds(self.triplet_list)

        k_array = self._to_array_1d(k, len(self.triplet_list))
        theta_array = self._to_array_1d(theta_0, len(self.triplet_list))

        energy = "kT * angK * 0.5 * (theta - angT0)^2"
        angle_force = mm.CustomAngleForce(energy)
        angle_force.setForceGroup(force_group)
        angle_force.addGlobalParameter("kT", self.sim_object.kT)
        angle_force.addPerAngleParameter("angK")
        angle_force.addPerAngleParameter("angT0")

        for i, (a, b, c) in enumerate(self.triplet_list):
            angle_force.addAngle(int(a), int(b), int(c), (float(k_array[i]), float(theta_array[i])))

        return angle_force
    
    def add_polynomial_repulsive(self, sim_object, trunc=3.0, radiusMult=1.0, name="polynomial_repulsive"):
        """
        Adds a soft repulsive polynomial potential between all particles.

        The potential:
        - Is flat until r ≈ 0.7 (relative to REPsigma)
        - Decays smoothly to 0 at r = REPsigma
        - Has finite energy at r = 0 (equal to `trunc` × kT)

        Based on: https://gist.github.com/mimakaev/0327bf6ffe7057ee0e0625092ec8e318

        Parameters
        ----------
        sim_object : Simulation
            Must have attributes: N (int), kT (float), conlen (float).
        trunc : float
            Repulsion strength at r = 0 (in kT units).
        radiusMult : float
            Multiplier on `sim_object.conlen` to define the cutoff radius.
        name : str
            Descriptive name for the force.

        Returns
        -------
        CustomNonbondedForce
        """

        # Define cutoff radius in reduced units
        radius = sim_object.conlen * radiusMult
        energy_expr = (
            "rsc12 * (rsc2 - 1.0) * REPe / emin12 + REPe;"
            "rsc12 = rsc4 * rsc4 * rsc4;"
            "rsc4 = rsc2 * rsc2;"
            "rsc2 = rsc * rsc;"
            "rsc = r / REPsigma * rmin12;"
        )

        force = mm.CustomNonbondedForce(energy_expr)
        force.setCutoffDistance(radius)
        force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffNonPeriodic)
        force.setForceGroup(3)  # Optional force group for repulsion

        # Global parameters
        force.addGlobalParameter("REPe", trunc * sim_object.kT)
        force.addGlobalParameter("REPsigma", radius)
        force.addGlobalParameter("emin12", 46656.0 / 823543.0)        # For x^12*(x²−1)
        force.addGlobalParameter("rmin12", np.sqrt(6.0 / 7.0))         # Scales distance into domain

        # Add particles
        for _ in range(sim_object.N):
            force.addParticle(())

        return force

    
    def add_spherical_confinement(
        self,
        sim_object,
        r="density",           # radius in reduced units or "density"
        k=5.0,                 # stiffness in kT / unit_length
        density=0.3,           # density for automatic radius estimation
        center=[0.0, 0.0, 0.0],# center of the sphere in reduced coordinates
        invert=False,          # exclude from sphere instead of confining
        particles=None,        # list of particle indices, or None for all
        name="spherical_confinement"
    ):
        """
        Constrain particles to be within (or outside) a sphere.

        Parameters
        ----------
        sim_object : Simulation
            Must have `N` (int), `kT` (float), `conlen` (float), and optionally `verbose`.
        r : float or "density"
            Confinement radius. If "density", computed from density and particle count.
        k : float
            Stiffness of the wall (in kT / conlen).
        density : float
            Density to use for automatic radius computation (in particles per unit volume).
        center : list of float
            Center of the sphere (3 values, in reduced coordinates).
        invert : bool
            If True, exclude particles from the sphere.
        particles : list of int, optional
            Which particles to apply the confinement to. Defaults to all.
        name : str
            Optional name for the force.

        Returns
        -------
        CustomExternalForce
            The spherical confinement potential.
        """
        # Calculate radius from density if requested
        if r == "density":
            r = (3 * sim_object.N / (4 * np.pi * density)) ** (1.0 / 3.0)

        if getattr(sim_object, "verbose", False):
            print(f"[spherical_confinement] radius = {r:.3f} (reduced units)")

        # Energy expression in reduced units
        energy_expr = (
            "step(invert_sign*(r-aa)) * kb * (sqrt((r-aa)^2 + t^2) - t); "
            "r = sqrt((x-x0)^2 + (y-y0)^2 + (z-z0)^2 + tt^2)"
        )

        force = mm.CustomExternalForce(energy_expr)

        # Add particles
        particles = range(sim_object.N) if particles is None else particles
        for i in particles:
            force.addParticle(int(i), [])

        # Parameters (no units)
        force.addGlobalParameter("kb", k * sim_object.kT)
        force.addGlobalParameter("aa", r - 1.0 / k)
        force.addGlobalParameter("t", (1.0 / k) / 10.0)
        force.addGlobalParameter("tt", 0.01)
        force.addGlobalParameter("invert_sign", -1.0 if invert else 1.0)

        # Center of confinement sphere
        force.addGlobalParameter("x0", center[0])
        force.addGlobalParameter("y0", center[1])
        force.addGlobalParameter("z0", center[2])

        sim_object.sphericalConfinementRadius = r  # for bookkeeping

        return force

    
    def add_nonbonded_pair_potential(
        self,
        sim_object,
        interactionMatrix,
        monomerTypes,
        rCutoff=1.8,
        name="custom_sticky_force"
    ):
        """
        Implements a sticky potential between monomer types.

        U_rep(r) = 5 * (1 + rRep^12 * (rRep^2 - 1) * c1)      for r < 1
        U_att(r) = -ε * (1 + rAtt^12 * (rAtt^2 - 1) * c1)     for 1 <= r < rCutoff
        ε is set by interactionMatrix[type1, type2]

        Parameters
        ----------
        sim_object : Simulation
            Must have attributes: N (int), conlen (float), kT (float).
        interactionMatrix : ndarray
            Symmetric matrix of ε values (float) between monomer types.
        monomerTypes : ndarray
            Array of length N assigning type index to each monomer.
        rCutoff : float
            Cutoff distance in reduced units (default 1.8).
        name : str
            Name for the force.

        Returns
        -------
        CustomNonbondedForce
        """

        Ntypes = max(monomerTypes) + 1
        if interactionMatrix.shape[0] < Ntypes or interactionMatrix.shape[1] < Ntypes:
            raise ValueError(f"Interaction matrix must cover all {Ntypes} types.")
        if not np.allclose(interactionMatrix.T, interactionMatrix):
            raise ValueError("Interaction matrix must be symmetric.")

        # Identify all interacting type pairs
        indexpairs = [(i, j) for i in range(Ntypes) for j in range(Ntypes) if interactionMatrix[i, j] != 0]

        # Constants
        c1 = (7.0 / 6.0) ** 6 * 7.0
        c2 = np.sqrt(6.0 / 7.0)

        # Construct energy expression
        energy = (
            "step(1.0 - r) * lambda_sticky * eRep + step(r - 1.0) * step(rCutoff - r) * lambda_sticky * eAttr;"
            "eRep = 5 * (1 + rRep12 * (rRep2 - 1) * c1);"
            "rRep12 = rRep4 * rRep4 * rRep4;"
            "rRep4 = rRep2 * rRep2;"
            "rRep2 = rRep * rRep;"
            "rRep = r * c2;"
            "eAttr = "
        )

        if indexpairs:
            terms = [f"delta(type1-{i})*delta(type2-{j})*INT_{i}_{j}" for i, j in indexpairs]
            energy += f"-1 * ({'+'.join(terms)}) * (1 + rAtt12 * (rAtt2 - 1) * c1);"
        else:
            energy += "0;"  # No attractions

        energy += (
            "rAtt12 = rAtt4 * rAtt4 * rAtt4;"
            "rAtt4 = rAtt2 * rAtt2;"
            "rAtt2 = rAtt * rAtt;"
            "rAtt = ((r - 1.4)/0.4) * c2;"
        )

        # Create force
        force = mm.CustomNonbondedForce(energy)
        force.setCutoffDistance(rCutoff * sim_object.conlen)
        force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffNonPeriodic)
        force.setForceGroup(2)  # Optional: assign to group 2 for analysis

        # Global parameters
        force.addGlobalParameter("rCutoff", rCutoff)
        force.addGlobalParameter("c1", c1)
        force.addGlobalParameter("c2", c2)
        force.addGlobalParameter("lambda_sticky", 1.0)

        for i, j in indexpairs:
            param_name = f"INT_{i}_{j}"
            force.addGlobalParameter(param_name, interactionMatrix[i, j])

        # Per-particle parameter
        force.addPerParticleParameter("type")
        for t in monomerTypes:
            force.addParticle([float(t)])

        return force


    @staticmethod
    def _generate_bonds(sim_object, chains, extra_bonds=None):
        """
        Generate list of bonds from chain definitions.
        
        Parameters
        ----------
        sim_object : Simulation
            Simulation object.
        chains : list of tuples
            List of (start, end, isRing) tuples.
        extra_bonds : list of tuples, optional
            Additional bonds to include.
            
        Returns
        -------
        numpy.ndarray
            Array of bond pairs.
        """
        bonds_list = [] if extra_bonds is None else [tuple(b) for b in extra_bonds]
        for start, end, is_ring in chains:
            end = sim_object.N if end is None else end
            bonds_list.extend([(j, j + 1) for j in range(start, end - 1)])
            if is_ring:
                bonds_list.append((start, end - 1))
        return np.array(bonds_list, dtype=int)

    @staticmethod
    def _generate_triplets(sim_object, chains, extra_triplets=None):
        """
        Generate list of angle triplets from chain definitions.
        
        Parameters
        ----------
        sim_object : Simulation
            Simulation object.
        chains : list of tuples
            List of (start, end, isRing) tuples.
        extra_triplets : list of tuples, optional
            Additional triplets to include.
            
        Returns
        -------
        numpy.ndarray
            Array of angle triplets.
        """
        triplets_list = [] if extra_triplets is None else [tuple(t) for t in extra_triplets]
        for start, end, is_ring in chains:
            end = sim_object.N if end is None else end
            triplets_list.extend([(j - 1, j, j + 1) for j in range(start + 1, end - 1)])
            if is_ring:
                triplets_list.append((end - 2, end - 1, start))
                triplets_list.append((end - 1, start, start + 1))
        return np.array(triplets_list, dtype=int)

    @staticmethod
    def _to_array_1d(val, length):
        return np.full(length, val) if np.isscalar(val) else np.asarray(val)

    @staticmethod
    def _check_angle_bonds(triplets):
        """
        Check for duplicate angle triplets.
        
        Parameters
        ----------
        triplets : array-like
            List of angle triplets to check.
            
        Raises
        ------
        ValueError
            If duplicate triplets are found.
        """
        seen = set()
        for t in triplets:
            # Convert numpy array to tuple for hashing
            t_tuple = tuple(t)
            if t_tuple in seen:
                raise ValueError(f"Duplicate angle triplet found: {t}")
            seen.add(t_tuple)

In [56]:
#test

Chromosome_sizes=[10]
chains = [(0,10,False)]
N=sum(Chromosome_sizes)
density=0.33

box_length = (N/density) ** (1/3.)
monomer_positions = create_random_walk(step_size=1,N=N)

sim = Simulation(
    integrator_type="variableLangevin",
    temperature=1.0,
    gamma=0.1,
    timestep=0.005,
    platform="CPU",
    N=N
)

sim.set_positions(monomer_positions)

chromosome = Chromosome(N, chains, sim)

harmonic_bond_force = chromosome.add_harmonic_bond()
angle_force = chromosome.add_angle_force()
spherical_confinement_force = chromosome.add_spherical_confinement(sim)
polynomial_repulsive_force = chromosome.add_polynomial_repulsive(sim)

sim.add_force(harmonic_bond_force)
sim.add_force(angle_force)
sim.add_force(spherical_confinement_force)
sim.add_force(polynomial_repulsive_force)

#must be called after all forces are added
sim.create_context()
sim.set_velocities()

#minimze energy
mm.LocalEnergyMinimizer.minimize(sim.context, tolerance=10.0, maxIterations=10000)


In [57]:
for _ in range(10):  # Do 10 blocks
    sim.run_simulation_block(100)  # Of 100 timesteps each. Data is saved automatically.
sim.print_stats() 

INFO:root:block    0 pos[1]=[0.9 0.4 -1.2] t=3.4τ kin=0.09 pot=0.04 SPS=2684
INFO:root:block    1 pos[1]=[2.2 1.2 -0.6] t=7.1τ kin=0.05 pot=0.03 SPS=2974
INFO:root:block    2 pos[1]=[1.4 0.2 1.2] t=11.3τ kin=0.04 pot=0.03 SPS=2974
INFO:root:block    3 pos[1]=[0.0 -0.1 1.3] t=15.6τ kin=0.03 pot=0.03 SPS=3014
INFO:root:block    4 pos[1]=[0.5 0.1 1.5] t=20.4τ kin=0.02 pot=0.01 SPS=2997
INFO:root:block    5 pos[1]=[0.6 0.4 2.0] t=25.7τ kin=0.02 pot=0.01 SPS=3061
INFO:root:block    6 pos[1]=[0.3 1.1 1.5] t=31.6τ kin=0.01 pot=0.01 SPS=3067
INFO:root:block    7 pos[1]=[-0.2 1.5 1.0] t=37.0τ kin=0.03 pot=0.01 SPS=3144
INFO:root:block    8 pos[1]=[-0.5 1.6 0.7] t=41.4τ kin=0.03 pot=0.01 SPS=2980
INFO:root:block    9 pos[1]=[-0.7 1.5 0.7] t=46.1τ kin=0.02 pot=0.02 SPS=3016



Simulation Statistics:
Current block: 10
Total steps: 1000
Simulation time: 46.14τ
Kinetic energy: 0.02 kT/particle
Potential energy: 0.02 kT/particle
Total energy: 0.04 kT/particle


In [45]:
# platforms = mm.Platform.findPlatform()
# print(platforms)

for i in range(mm.Platform.getNumPlatforms()):
    platform = mm.Platform.getPlatform(i)
    print(f"Platform {i}: {platform.getName()}")


Platform 0: Reference
Platform 1: CPU
Platform 2: OpenCL
